In [ ]:
# Not needed if tinc-python is installed
import sys
sys.path.append('../tinc-python')

# Graph update example

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from parameter_server import *
pserver = ParameterServer()

Parameter server started on ('127.0.0.1', 9011)
Command server: Serving on ('127.0.0.1', 14001)


In [3]:
param = Parameter("param")
pserver.register_parameter(param)

In [4]:
from disk_buffer_writer import *
dataWriter = DiskBufferWriter('graph','output.png', 'bin/')
dataWriter.expose_to_network(pserver)
dataWriter.allow_cache(10)

In [ ]:
dataWriter.cleanup_cache()

In [5]:
import random
import matplotlib.pyplot as plt
import threading

def update_graph(parameter_value):
    print("Parameter value " + str(parameter_value))
    data = [random.random() * parameter_value for i in range(10)]
    with threading.Lock():
        fname = dataWriter.get_filename_for_writing()
        f = plt.figure()
        plt.title("Random numbers with range 0->" + str(parameter_value))
        plt.plot(data)
        print("Update " + fname)
        plt.savefig(fname)
        plt.close()
        f.clf()
        del f
        dataWriter.done_writing_file(fname)

In [6]:
update_graph(1)

Parameter value 1
Update bin/output_0.png


In [7]:
param.register_callback(update_graph)

In [ ]:
pserver.monitor_server()

In [ ]:
pserver.stop()

In [ ]:
param.value

In [ ]:
pwd